In [2]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow.keras.backend as K
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import pandas as pd
import os
import datetime
import argparse
from VAE_functions import *
from NILM_functions import *
import pickle
from scipy.stats import norm
from keras.utils.vis_utils import plot_model
import logging
import json

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
thr_house_2 = { "FridgeFreezer" : 50,
                "WashingMachine" : 20,
                "DishWasher" : 100,
                "Kettle" : 100,
                "Microwave" : 200,
               "Toaster" :100,
                 "fan" : 5,
                  "computerMonitor" : 60}

In [4]:
parser = argparse.ArgumentParser()
parser
parser.add_argument("--gpu", default=0, type=int, help="GPU to use")
parser.add_argument("--config", default="Config/House_2/Microwave_VAE.json", type=str, help="Path to the config file")
parser.add_argument("--time", default="", type=str, help="Folder name containing runs")
parser.add_argument("--save_pred", default=True, type=bool, help="Save y_pred_all")
parser.add_argument("--previous", default=1, type=int, help="Select previous experiment")

_StoreAction(option_strings=['--previous'], dest='previous', nargs=None, const=None, default=1, type=<class 'int'>, choices=None, help='Select previous experiment', metavar=None)

In [5]:
a = parser.parse_args(args=[])
print(a)
with open(a.config) as data_file:
    nilm = json.load(data_file)

Namespace(config='Config/House_2/Microwave_VAE.json', gpu=0, previous=1, save_pred=True, time='')


In [6]:
name = "NILM_Disag_{}".format(nilm["appliance"])
previous = a.previous

if a.time == "":
    files_and_directories = os.listdir("../../data/NILM/{}/ukdale/{}/logs/model/House_{}/".format(name, nilm["model"], nilm["dataset"]["test"]["house"][0]))
    #files_and_directories = os.listdir("{}/ukdale/{}/logs/model/House_{}/".format(name, nilm["model"], nilm["dataset"]["test"]["house"][0]))
    time = np.sort(files_and_directories)[-previous]
else:
    time = a.time

In [52]:
x_total = []
y_total_pred = []
y_total_true = []

#data = np.load("../../data/NILM/{}/ukdale/{}/logs/model/House_{}/{}/pred_{}.npy".format(name, nilm["model"], nilm["dataset"]["test"]["house"][0], time,1),encoding = "latin1",allow_pickle=True)
#data = np.load("{}/ukdale/{}/logs/model/House_{}/{}/pred_{}.npy".format(name, nilm["model"], nilm["dataset"]["test"]["house"][0], time, r),encoding = "latin1",allow_pickle=True)
data = np.load("../../data/NILM/NILM_Disag_Microwave/ukdale/VAE/logs/model/House_12/20220715-172909/pred_1.npy")
x_total = data[0]
y_total_pred = data[1]
y_total_true = data[2]
def acc_Power1(x_pred, x_test, c_test=0, app_ratio=0, disaggregation=False):
    x_pred = y_total_pred
    x_test = y_total_true
    if disaggregation:
        Pest = np.sum(x_pred, axis=1).reshape([-1,1])
        print(Pest)
        Preal = np.sum(x_test, axis=1).reshape([-1,1])
        print(Preal)
        acc_P = ((np.abs(Pest - Preal)/(2*Preal))*-1)+1
        print(acc_P)
        acc_P = np.nan_to_num(acc_P)
        print(acc_P)
        acc_P_tot = np.mean(acc_P[acc_P>0])
        acc_P_app = acc_P.reshape(-1)
    else:
        M_ratio = (np.tile(app_ratio, [c_test.shape[0],1])*c_test)
        print(M_ratio)
        Pest = np.sum(x_pred, axis=1).reshape([-1,1])
        print(Pest)
        Pest = (np.tile(Pest, [1,c_test.shape[1]])*M_ratio)
        print(Pest)
        Preal = np.sum(x_test, axis=1).reshape([-1,1])
        print(Preal)
        Preal = (np.tile(Preal, [1,c_test.shape[1]])*M_ratio)
        print(Preal)
        acc_P = ((np.abs(Pest - Preal)/(2*Preal))*-1)+1
        print(acc_P)
        acc_P = np.nan_to_num(acc_P)
        acc_P_tot = acc_P.sum(axis=1).mean()
        acc_P_app = acc_P.sum(axis=0)/c_test.sum(axis=0)
    print(acc_P_tot,acc_P_app,acc_P)

acc_Power1(y_total_pred, y_total_true, disaggregation=True)

[[3130106.27150726]]
[[619969.47222222]]
[[-1.02440355]]
[[-1.02440355]]
nan [-1.02440355] [[-1.02440355]]


In [42]:
MAE_run = []
ACC_run =[]
PR_run = []
RE_run = []
F1_run = []
SAE_run = []
FEE_run = []
#for r in range(1, nilm["run"]+1):
for r in range(1, 11):   
    x_total = []
    y_total_pred = []
    y_total_true = []
    
    #data = np.load("../../data/NILM/{}/ukdale/{}/logs/model/House_{}/{}/pred_{}.npy".format(name, nilm["model"], nilm["dataset"]["test"]["house"][0], time, r),encoding = "latin1",allow_pickle=True)
    data = np.load("../../data/NILM/NILM_Disag_Microwave/ukdale/VAE/logs/model/House_12/20220714-120424/pred_1.npy")
    x_total = data[0]
    y_total_pred = data[1]
    y_total_true = data[2]
  
    MAE_tot, MAE_app, MAE = MAE_metric(y_total_pred, y_total_true, disaggregation=True, only_power_on=False)
    acc_P_tot, acc_P_app, acc_P = acc_Power(y_total_pred, y_total_true, disaggregation=True)
    # PR_app = PR_metric(y_total_pred, y_total_true, thr=thr_house_2[nilm["appliance"]])
    # RE_app = RE_metric(y_total_pred, y_total_true, thr=thr_house_2[nilm["appliance"]])
    # F1_app = F1_metric(y_total_pred, y_total_true, thr=thr_house_2[nilm["appliance"]])
    PR_app = PR_metric(y_total_pred, y_total_true, thr=thr_house_2["computerMonitor"])
    RE_app = RE_metric(y_total_pred, y_total_true, thr=thr_house_2["computerMonitor"])
    F1_app = F1_metric(y_total_pred, y_total_true, thr=thr_house_2["computerMonitor"])
    SAE_app = SAE_metric(y_total_pred, y_total_true)
    FEE_app = FEE_metric(y_total_pred, y_total_true)
    print(FEE_app)
    if (np.isnan(acc_P_tot)) or (F1_app[0] == 0):
        print("Error Detected")
    else:
        MAE_run.append(MAE_tot)
        ACC_run.append(acc_P_tot)
        PR_run.append(PR_app[0])
        RE_run.append(RE_app[0])
        F1_run.append(F1_app[0])
        SAE_run.append(SAE_app[0])
        FEE_run.append(FEE_app[0])

print(np.mean(MAE_run), np.std(MAE_run))
print(np.mean(ACC_run), np.std(ACC_run))
print(np.mean(PR_run), np.std(PR_run))
print(np.mean(RE_run), np.std(RE_run))
print(np.mean(F1_run), np.std(F1_run))
print(np.mean(SAE_run), np.std(SAE_run))
print(np.mean(FEE_run), np.std(FEE_run))

#np.save("../../data/NILM/{}/ukdale/{}/logs/model/House_{}/{}/results_median.npy".format(name, nilm["model"], nilm["dataset"]["test"]["house"][0], time), [MAE_run, ACC_run, PR_run, RE_run, F1_run, SAE_run, FEE_run])

44.29329509544597
nan
0.03819808406997797


KeyboardInterrupt: 